# **Audio Classification using MFCC features**
#Task: Gender Prediction using ML and DL 


In [1]:
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.4 MB/s eta 0:00:00


# Data Preparation

Library dataset from Huggingface 

In [2]:
import random
random.seed(10)

In [3]:
import librosa
import numpy as np
import pandas as pd
from datasets import load_dataset
import timeit

In [4]:
dataset = load_dataset('common_voice', 'ta') # loads hindi corpus

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating train split:   0%|          | 0/2009 [00:00<?, ? examples/s]

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating test split:   0%|          | 0/1781 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1779 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/7428 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/12652 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/594 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/ta/6.1.0/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf. Subsequent calls will reuse this data.


  0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
dataset = load_dataset('common_voice', 'ta') # loads tamil corpus
train = dataset["train"]

  0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2009
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1781
    })
    validation: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1779
    })
    other: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 7428
    })
    validated: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 12652
    })
    invalidated: Dataset({
        feature

In [7]:
set(dataset['train']['age'])

{'thirties', 'twenties'}

In [8]:
set(dataset['test']['age'])

{'', 'fifties', 'fourties', 'teens', 'thirties', 'twenties'}

In [86]:
set(dataset['validation']['age'])

{'', 'fifties', 'fourties', 'thirties', 'twenties'}

As we see a difference , we now choose to concatinate these 3 dataframes.

In [9]:
train = dataset['train'].to_pandas()
test = dataset['test'].to_pandas()
valid = dataset['validation'].to_pandas()

In [10]:
dataset_combined = pd.concat([train,test,valid])

In [11]:
set(dataset_combined['age']), dataset_combined.shape

({'', 'fifties', 'fourties', 'teens', 'thirties', 'twenties'}, (5569, 11))

In [12]:
sum((dataset_combined['age'] == ''))

585

In [13]:
index_names = dataset_combined[ dataset_combined['age'] == '' ].index
  
# drop these row indexes as they are empty 
dataset_combined.drop(index_names, inplace = True)

In [14]:
sum((dataset_combined['age'] == ''))

0

In [66]:
from sklearn.utils import shuffle
train = shuffle(dataset_combined,random_state=25)

In [67]:
train_files, train_labels = train["path"], train["age"]
train_df = pd.DataFrame.from_dict({"path": train_files, "label": train_labels})
train_files, train_labels = list(train_df["path"]), list(train_df["label"])

In [68]:
# Get the count of each unique element
counts = [(x, train_labels.count(x)) for x in set(train_labels)]

# Print the count of each element
for x, count in counts:
    print(f"{x}: {count}")

thirties: 1663
twenties: 1995
fourties: 125
fifties: 74
teens: 11


# Feature Extraction - MFCC 

In [69]:
x , sr = librosa.load(train_files[0], sr=16000)

In [70]:
mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=20)

In [71]:
mfcc.shape

(20, 136)

In [72]:
mfcc.reshape(-1,1)[:1000] # flattening to get a single feature vector


array([[-5.06512878e+02],
       [-4.48736633e+02],
       [-4.85438507e+02],
       [-5.52430603e+02],
       [-5.88118164e+02],
       [-5.98439331e+02],
       [-5.98165100e+02],
       [-5.91606873e+02],
       [-5.90010071e+02],
       [-5.69968811e+02],
       [-5.06314911e+02],
       [-4.27192352e+02],
       [-2.44973053e+02],
       [-1.60402771e+02],
       [-1.51181747e+02],
       [-1.52030777e+02],
       [-1.32591904e+02],
       [-1.30768570e+02],
       [-1.58100906e+02],
       [-1.91707443e+02],
       [-1.63266586e+02],
       [-1.54715057e+02],
       [-1.56236420e+02],
       [-1.82580994e+02],
       [-1.86106583e+02],
       [-1.80242920e+02],
       [-2.25842285e+02],
       [-2.40561050e+02],
       [-1.93769135e+02],
       [-1.73391388e+02],
       [-2.24330246e+02],
       [-2.81657135e+02],
       [-3.47293579e+02],
       [-4.36214020e+02],
       [-4.69486359e+02],
       [-3.92064972e+02],
       [-3.16924133e+02],
       [-3.06195160e+02],
       [-3.3

In [73]:
def get_mfcc_features(files, labels):
    temp_feats = list()
    temp_labels = list()

    for file, label in zip(files, labels):
        try:
            x, sr = librosa.load(file, sr=16000) # load the audio file
            mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=20) # extract the mfcc features
            temp_feats.append(mfcc.reshape(-1)[:1000]) # flattening to get a single feature vector
            # fixing to get only the 1000 samples to have common feature size though the audio length differs
            if label == "twenties":
              temp_labels.append(0)
            elif label == "thirties":
              temp_labels.append(1)
            elif label == "fourties":
              temp_labels.append(2)
            elif label == "fifties":
              temp_labels.append(3)
            elif label == "teens":
              temp_labels.append(4)


        except:
            pass
    return np.array(temp_feats), np.array(temp_labels)

In [74]:
train_features, train_labels = get_mfcc_features(train_files, train_labels)

In [75]:
train_features.shape

(3868, 1000)

In [76]:
train_labels.shape

(3868,)

# Conventional Machine Learning - Random Forest

In [77]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [78]:
train_labels

array([0, 0, 0, ..., 1, 0, 1])

In [79]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

t0 = timeit.default_timer()

scaler = MinMaxScaler(feature_range=(0, 1))
train_features = scaler.fit_transform(train_features)

# One-hot encoding of train_labels
label_binarizer = LabelBinarizer()
train_labels = label_binarizer.fit_transform(train_labels)

# split data into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.20, random_state=1150)

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=75, random_state=42)

# Train the classifier
rf.fit(X_train, y_train)

# Predict on the test set
y_pred = rf.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy: %.4f' % accuracy)

# Calculate the confusion matrix
y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test_class, y_pred_class)
print("Confusion Matrix:")
print(cm)

run_time = timeit.default_timer() - t0
print('Example run in %.3f s' % run_time)


Test accuracy: 0.6021
Confusion Matrix:
[[335  52   0   0   0]
 [156 175   0   0   0]
 [ 21   7   0   0   0]
 [ 21   2   0   0   0]
 [  4   1   0   0   0]]
Example run in 8.396 s


We can see that the model performs well on classifying class_0 (twenties) and class_1 (thirties) but struggles with the other classes (fourties, fifties, and teens). It has not predicted any samples for classes 2, 3, and 4.

In [80]:
# other classification metrics 
from sklearn.metrics import classification_report
target_names=['twenties','thirties','fourties','fifties', 'teens']
print(classification_report(y_test_class, y_pred_class, target_names=target_names))

              precision    recall  f1-score   support

    twenties       0.62      0.87      0.73       387
    thirties       0.74      0.53      0.62       331
    fourties       0.00      0.00      0.00        28
     fifties       0.00      0.00      0.00        23
       teens       0.00      0.00      0.00         5

    accuracy                           0.66       774
   macro avg       0.27      0.28      0.27       774
weighted avg       0.63      0.66      0.63       774



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The model has good precision and recall for the twenties and thirties classes, with F1-scores of 0.73 and 0.62, respectively. However, for the other classes (fourties, fifties, and teens), the model has not made any correct predictions, resulting in precision, recall, and F1-score of 0.



# Deep Learning - Simple Neural Network

In [31]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder

* Tensorflow - end-to-end open source platform for machine learning
* Focused on training and inference of deep neural networks
* For more information refer this [link](https://www.tensorflow.org/)

* Keras -  interface for artificial neural networks libraries
* high-level API
* For more information refer this (link)[https://keras.io/]

In [32]:
train_features.shape[1]

1000

In [33]:

# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(100, input_dim=train_features.shape[1], activation='relu')) # one hidden layer
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(5, activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [35]:
model = create_baseline()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [81]:

model.fit(X_train, y_train, epochs=100)

Epoch 1/100
97/97 [==============================] - 0s 3ms/step - loss: 0.6958 - accuracy: 0.7085
Epoch 2/100
97/97 [==============================] - 0s 3ms/step - loss: 0.6627 - accuracy: 0.7133
Epoch 3/100
97/97 [==============================] - 0s 4ms/step - loss: 0.6498 - accuracy: 0.7282
Epoch 4/100
97/97 [==============================] - 1s 5ms/step - loss: 0.6534 - accuracy: 0.7204
Epoch 5/100
97/97 [==============================] - 0s 5ms/step - loss: 0.6349 - accuracy: 0.7334
Epoch 6/100
97/97 [==============================] - 1s 5ms/step - loss: 0.6437 - accuracy: 0.7295
Epoch 7/100
97/97 [==============================] - 1s 5ms/step - loss: 0.6311 - accuracy: 0.7279
Epoch 8/100
97/97 [==============================] - 0s 4ms/step - loss: 0.6337 - accuracy: 0.7237
Epoch 9/100
97/97 [==============================] - 0s 4ms/step - loss: 0.6295 - accuracy: 0.7356
Epoch 10/100
97/97 [==============================] - 0s 3ms/step - loss: 0.6490 - accuracy: 0.7156
Epoch 11/

In [82]:
loss_and_metrics = model.evaluate(X_test, y_test)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

25/25 [==============================] - 0s 3ms/step - loss: 0.9326 - accuracy: 0.6667
Loss =  0.9325844645500183
Accuracy =  0.6666666865348816


In [83]:
y_pred =  model.predict(X_test)

25/25 [==============================] - 0s 2ms/step


In [84]:
# Calculate the confusion matrix
y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test_class, y_pred_class)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[302  75   5   5   0]
 [119 198   7   7   0]
 [ 11   8   6   3   0]
 [ 10   4   0   9   0]
 [  3   1   0   0   1]]


Better classification of lst 3 classes 

In [85]:
# other classification metrics 
from sklearn.metrics import classification_report
target_names=['twenties','thirties','fourties','fifties', 'teens']
print(classification_report(y_test_class, y_pred_class, target_names=target_names))


              precision    recall  f1-score   support

    twenties       0.68      0.78      0.73       387
    thirties       0.69      0.60      0.64       331
    fourties       0.33      0.21      0.26        28
     fifties       0.38      0.39      0.38        23
       teens       1.00      0.20      0.33         5

    accuracy                           0.67       774
   macro avg       0.62      0.44      0.47       774
weighted avg       0.67      0.67      0.66       774



# INFERENCE

Though the accuracy remains the same , we can see that the DL model actually tries to classifies fourties,fifties,teens. It has got a decent amount correct . But the ML method just classfies the maximum ones better.

- The data is not evently distributed , maybe performing oversampling of last 3 classes might improve the performance . 
